In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!gcloud init --skip-diagnostics

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You must log in to continue. Would you like to log in (Y/n)?  y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&prompt=select_account&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&access_type=offline



In [0]:
from google.colab import auth
auth.authenticate_user()

#Export

In [0]:
import os
import sys

# For running inference on the TF-Hub module.
import tensorflow as tf

In [0]:
!git clone https://github.com/tensorflow/models.git /content/tf
!cd /content/tf/research; protoc object_detection/protos/*.proto --python_out=.
!cd /content/tf/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python /content/tf/research/object_detection/builders/model_builder_test.py

Cloning into '/content/tf'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 26576 (delta 3), reused 2 (delta 0), pack-reused 26562
Receiving objects: 100% (26576/26576), 508.83 MiB | 33.40 MiB/s, done.
Resolving deltas: 100% (16058/16058), done.
Checking out files: 100% (2978/2978), done.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.099s

OK (skipped=1)


In [0]:
sys.path.append('tf/research')
sys.path.append('tf/research/slim')
import object_detection

In [0]:
os.environ["CONFIG_FILE"] = "pipeline.config"
os.environ["CHECKPOINT_PATH"] = "mobilenetv1/model.ckpt-7000"
os.environ["OUTPUT_DIR"] = "tflite/"

In [0]:
import tensorflow as tf
from google.protobuf import text_format
from object_detection import export_tflite_ssd_graph_lib
from object_detection.protos import pipeline_pb2

In [0]:
flags = tf.app.flags
flags.DEFINE_string('output_directory', None, 'Path to write outputs.')
flags.DEFINE_string(
    'pipeline_config_path', None,
    'Path to a pipeline_pb2.TrainEvalPipelineConfig config '
    'file.')
flags.DEFINE_string('trained_checkpoint_prefix', None, 'Checkpoint prefix.')
flags.DEFINE_integer('max_detections', 10,
                     'Maximum number of detections (boxes) to show.')
flags.DEFINE_integer('max_classes_per_detection', 1,
                     'Number of classes to display per detection box.')
flags.DEFINE_integer(
    'detections_per_class', 100,
    'Number of anchors used per class in Regular Non-Max-Suppression.')
flags.DEFINE_bool('add_postprocessing_op', True,
                  'Add TFLite custom op for postprocessing to the graph.')
flags.DEFINE_bool(
    'use_regular_nms', False,
    'Flag to set postprocessing op to use Regular NMS instead of Fast NMS.')
flags.DEFINE_string(
    'config_override', '', 'pipeline_pb2.TrainEvalPipelineConfig '
    'text proto to override pipeline_config_path.')

FLAGS = flags.FLAGS

In [0]:
def main(argv):
  del argv  # Unused.
  flags.mark_flag_as_required('output_directory')
  flags.mark_flag_as_required('pipeline_config_path')
  flags.mark_flag_as_required('trained_checkpoint_prefix')

  pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

  with tf.gfile.GFile(FLAGS.pipeline_config_path, 'r') as f:
    text_format.Merge(f.read(), pipeline_config)
  text_format.Merge(FLAGS.config_override, pipeline_config)
  export_tflite_ssd_graph_lib.export_tflite_graph(
      pipeline_config, FLAGS.trained_checkpoint_prefix, FLAGS.output_directory,
      FLAGS.add_postprocessing_op, FLAGS.max_detections,
      FLAGS.max_classes_per_detection, FLAGS.use_regular_nms)

In [0]:
#!cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/ConfigFiles/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_colabTPU.config pipeline.config
!cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/ConfigFiles/ssd_mobilenet_v1_320x320_holoselecta_sync_colabTPU.config pipeline.config

In [0]:
!mkdir mobilenetv1
!gsutil -m cp gs://holoselecta_objectdetection/mobilenetv1/320/1000/* /content/mobilenetv1/

Copying gs://holoselecta_objectdetection/mobilenetv1/320/1000/model.ckpt-0.index...
Copying gs://holoselecta_objectdetection/mobilenetv1/320/1000/checkpoint...
Copying gs://holoselecta_objectdetection/mobilenetv1/320/1000/events.out.tfevents.1559220267.32d3e882bf3e...
Copying gs://holoselecta_objectdetection/mobilenetv1/320/1000/graph.pbtxt...
Copying gs://holoselecta_objectdetection/mobilenetv1/320/1000/model.ckpt-0.meta...
Copying gs://holoselecta_objectdetection/mobilenetv1/320/1000/model.ckpt-7000.data-00000-of-00001...
Copying gs://holoselecta_objectdetection/mobilenetv1/320/1000/model.ckpt-7000.meta...
Copying gs://holoselecta_objectdetection/mobilenetv1/320/1000/model.ckpt-7000.index...
Copying gs://holoselecta_objectdetection/mobilenetv1/320/1000/pipeline.config...
Copying gs://holoselecta_objectdetection/mobilenetv1/320/1000/model.ckpt-0.data-00000-of-00001...


In [0]:
FLAGS.pipeline_config_path = "pipeline.config"
FLAGS.trained_checkpoint_prefix = "/content/mobilenetv1/model.ckpt-7000"
FLAGS.output_directory = "tflite"

tf.app.run(main)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/mobilenetv1/model.ckpt-7000
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 199 variables.
INFO:tensorflow:Converted 199 variables to const ops.


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
!git clone https://github.com/tensorflow/tensorflow.git

Cloning into 'tensorflow'...
remote: Enumerating objects: 1715, done.
remote: Counting objects: 100% (1715/1715), done.
remote: Compressing objects: 100% (754/754), done.
remote: Total 597640 (delta 1094), reused 1235 (delta 959), pack-reused 595925
Receiving objects: 100% (597640/597640), 343.10 MiB | 27.08 MiB/s, done.
Resolving deltas: 100% (483525/483525), done.
Checking out files: 100% (17103/17103), done.


In [0]:
!cd tensorflow; /root/bin/bazel run -c opt tensorflow/lite/toco:toco -- --input_file=$OUTPUT_DIR/tflite_graph.pb --output_file=$OUTPUT_DIR/detect.tflite --input_shapes=1,320,320,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=QUANTIZED_UINT8 --mean_values=128 --std_values=128 --change_concat_input_ranges=false --allow_custom_ops